<a href="https://colab.research.google.com/github/Papa-Panda/random_thoughts/blob/main/ML_interview_two_tower_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn

class TwoTowerModel(nn.Module):
    def __init__(self, user_dim, item_dim, embedding_dim):
        super(TwoTowerModel, self).__init__()

        # User tower
        self.user_embedding = nn.Embedding(user_dim, embedding_dim)

        # Item tower
        self.item_embedding = nn.Embedding(item_dim, embedding_dim)

        # Fully connected layers for interaction
        self.fc1 = nn.Linear(embedding_dim * 2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.output_layer = nn.Linear(64, 1)

        # Activation function
        self.relu = nn.ReLU()

    def forward(self, user_input, item_input):
        # User tower
        user_embedded = self.user_embedding(user_input)
        user_embedded = user_embedded.view(user_embedded.size(0), -1)  # Flatten the user embeddings

        # Item tower
        item_embedded = self.item_embedding(item_input)
        item_embedded = item_embedded.view(item_embedded.size(0), -1)  # Flatten the item embeddings

        # Concatenate user and item embeddings
        merged = torch.cat((user_embedded, item_embedded), dim=1)

        # Fully connected layers
        x = self.relu(self.fc1(merged))
        x = self.relu(self.fc2(x))

        # Output layer
        output = self.output_layer(x)

        return output.squeeze()


In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# Sample data
user_data = torch.tensor([1, 2, 3, 4, 5])  # User IDs
item_data = torch.tensor([101, 102, 103, 104, 105])  # Item IDs
labels = torch.tensor([5.0, 4.0, 3.0, 4.5, 2.5])  # Ratings

# Hyperparameters
embedding_dim = 10
user_dim = max(user_data) + 1
item_dim = max(item_data) + 1
learning_rate = 0.001
epochs = 100

# Create model, loss function, and optimizer
model = TwoTowerModel(user_dim, item_dim, embedding_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Create DataLoader
dataset = TensorDataset(user_data, item_data, labels)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Training loop
for epoch in range(epochs):
    for batch_user, batch_item, batch_labels in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_user, batch_item)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

# Inference
user_input = torch.tensor([2])  # User ID for inference
item_input = torch.tensor([103])  # Item ID for inference

with torch.no_grad():
    prediction = model(user_input, item_input)

print(f"Prediction for User {user_input.item()} on Item {item_input.item()}: {prediction.item()}")


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/100, Loss: 25.113325119018555
Epoch 2/100, Loss: 7.805745601654053
Epoch 3/100, Loss: 7.23158073425293
Epoch 4/100, Loss: 4.651462554931641
Epoch 5/100, Loss: 5.96266508102417
Epoch 6/100, Loss: 3.467662811279297
Epoch 7/100, Loss: 15.481888771057129
Epoch 8/100, Loss: 12.163918495178223
Epoch 9/100, Loss: 1.446319341659546
Epoch 10/100, Loss: 0.8458327651023865
Epoch 11/100, Loss: 0.35092005133628845
Epoch 12/100, Loss: 4.9857940673828125
Epoch 13/100, Loss: 1.6962560415267944
Epoch 14/100, Loss: 2.904329538345337
Epoch 15/100, Loss: 0.7084870934486389
Epoch 16/100, Loss: 0.8219170570373535
Epoch 17/100, Loss: 0.0005193602992221713
Epoch 18/100, Loss: 0.1495336890220642
Epoch 19/100, Loss: 1.0776991844177246
Epoch 20/100, Loss: 0.10086991637945175
Epoch 21/100, Loss: 0.042469341307878494
Epoch 22/100, Loss: 0.017738601192831993
Epoch 23/100, Loss: 0.05337188020348549
Epoch 24/100, Loss: 0.007501512300223112
Epoch 25/100, Loss: 0.022048557177186012
Epoch 26/100, Loss: 0.0011361